In [5]:
import urllib.parse
import requests

In [1]:
def get_states():
    query_params = {    
            "lang": "en",
            'q' : [{"identifier":'level','value':'State'}],
            "keyword": "",
            "sort": "multiple_sort",
            "from": "0",
            "size": "70",
        }


    url = "https://api.myscheme.gov.in/search/v4/schemes/facets?"+urllib.parse.urlencode(query_params).replace("%27","%22")

    payload = {}
    headers = {
      'x-api-key': 'tYTy5eEhlu9rFjyxuCr7ra7ACp4dv1RH8gWuHTDc'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    states=[]
    for facet in response.json()['data']['facets']:
        if facet['identifier']=="beneficiaryState":
            states=facet['entries']
    return states

In [6]:
states=get_states()

In [7]:
states

[{'label': 'Gujarat', 'count': 263},
 {'label': 'Tamil Nadu', 'count': 235},
 {'label': 'Puducherry', 'count': 227},
 {'label': 'Haryana', 'count': 189},
 {'label': 'Madhya Pradesh', 'count': 183},
 {'label': 'Goa', 'count': 146},
 {'label': 'Rajasthan', 'count': 110},
 {'label': 'Bihar', 'count': 103},
 {'label': 'West Bengal', 'count': 101},
 {'label': 'Jharkhand', 'count': 95},
 {'label': 'Chhattisgarh', 'count': 90},
 {'label': 'Maharashtra', 'count': 85},
 {'label': 'Kerala', 'count': 73},
 {'label': 'Odisha', 'count': 73},
 {'label': 'Andhra Pradesh', 'count': 66},
 {'label': 'Himachal Pradesh', 'count': 66},
 {'label': 'Karnataka', 'count': 63},
 {'label': 'Meghalaya', 'count': 63},
 {'label': 'Dadra & Nagar Haveli and Daman & Diu', 'count': 50},
 {'label': 'Assam', 'count': 47},
 {'label': 'Delhi', 'count': 46},
 {'label': 'Jammu and Kashmir', 'count': 45},
 {'label': 'Uttar Pradesh', 'count': 45},
 {'label': 'Tripura', 'count': 40},
 {'label': 'Punjab', 'count': 37},
 {'label'

In [8]:
def get_slugs(state,size):
    query_params = {    
        "lang": "en",
        'q' : [{"identifier":'level','value':'State'},{'identifier':'beneficiaryState','value':state}],
        "keyword": "",
        "sort": "multiple_sort",
        "from": "0",
        "size": "70",
    }
    
    url = "https://api.myscheme.gov.in/search/v4/schemes?"+urllib.parse.urlencode(query_params).replace("%27","%22")

    payload = {}
    headers = {
      'x-api-key': 'tYTy5eEhlu9rFjyxuCr7ra7ACp4dv1RH8gWuHTDc'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    slugs=[]
    for item in response.json()['data']['hits']['items']:    
        slugs.append(item['fields']['slug'])
    return slugs

In [9]:
get_slugs("Gujarat","270")

['mmuy',
 'sspftws',
 'mmksy',
 'ssst-pts',
 'km',
 'casscsppnjg',
 'spy-gbocwwb',
 'sndaygbocwwb',
 'mbbsgbocwwb',
 'vcsgbocwwb',
 'hssgbocwwb',
 'masglwb',
 'smts-glwb',
 'lpas-glwb',
 'sfmcas-glwb',
 'dpgsslampgdsc',
 'faspalscf',
 'fmtaspsstc',
 'apdccsifdsgc',
 'apdccsscspscc',
 'paapwd',
 'sss-gujarat',
 'ftstbdp',
 'ma',
 'mssg',
 'agsmais',
 'agsmnsgst',
 'agmsmercgt',
 'gsrfas',
 'saepaep',
 'sfailfcs',
 'sfasisd',
 'faitfci',
 'faitsd',
 'sfalp',
 'setcmsuc',
 'samsmeis',
 'bauuyseat',
 'bauuyata',
 'samsmesme',
 'gtpaqc',
 'gtccs',
 'sfatm',
 'gtcisliu',
 'sfasiss',
 'apapusmc',
 'agrasp',
 'afccdwt',
 'uadtas',
 'mds',
 'fsss',
 'agr2fmsfotscst',
 'sseg10scp',
 'mta',
 'mysy',
 'agr4fmsscf',
 'asodgbocwwb',
 'eaphdsgbocwwb',
 'ggtwsgbocwwb',
 'adasgbocwwb',
 'saygbocwwb',
 'eas-glwb',
 'ggs-glwb',
 'ptascsss1112ss',
 'hliss-glwb',
 'mas-glwb',
 'pmsscs-g',
 'dpgsslalgsc',
 'ssscsitipcsvs',
 'casprescs']

In [10]:
import json
import re

def clean_text(text):
    """Clean text by removing extra spaces and newlines."""
    if not text:
        return ""
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Strip leading and trailing whitespace
    return text.strip()

def extract_text_from_children(children):
    """Extract text from children nodes recursively."""
    result = ""
    if not children:
        return result
    
    for child in children:
        if isinstance(child, dict):
            if "text" in child:
                text = child["text"]
                if child.get("bold"):
                    text = f"{text}"  # Keep bold text as is (would be bold in a word processor)
                result += text
            if "children" in child:
                result += extract_text_from_children(child["children"])
    
    return result

def parse_content(content, level=0):
    """Parse content nodes and return formatted text."""
    result = ""
    
    if isinstance(content, list):
        for item in content:
            if isinstance(item, dict):
                item_type = item.get("type", "")
                
                if item_type == "paragraph":
                    paragraph_text = extract_text_from_children(item.get("children", []))
                    if paragraph_text:
                        result += paragraph_text + "\n\n"
                
                elif item_type == "ol_list" or item_type == "ul_list":
                    result += parse_content(item.get("children", []), level+1)
                
                elif item_type == "list_item":
                    item_text = extract_text_from_children(item.get("children", []))
                    if item_text:
                        indent = "  " * (level-1)
                        result += f"{indent}• {item_text}\n"
                
                elif item_type == "align_justify":
                    result += parse_content(item.get("children", []), level)
                
                else:
                    if "children" in item:
                        result += parse_content(item["children"], level)
    
    return result

def json_to_document(json_str):
    """Convert JSON string to human-readable document."""
    try:
        # Parse JSON
        data = json.loads(json_str)
        
        # Extract scheme data
        scheme_data = data.get("pageProps", {}).get("schemeData", {}).get("en", {})
        basic_details = scheme_data.get("basicDetails", {})
        scheme_content = scheme_data.get("schemeContent", {})
        application_process = scheme_data.get("applicationProcess", [])
        eligibility_criteria = scheme_data.get("eligibilityCriteria", {})
        
        # Extract FAQ data
        faqs = data.get("pageProps", {}).get("faqs", {}).get("data", {}).get("en", {}).get("faqs", [])
        
        # Extract document requirements
        docs = data.get("pageProps", {}).get("docs", {}).get("data", {}).get("en", {}).get("documents_required", [])
        
        # Build the document
        document = ""
        
        # Title
        scheme_name = basic_details.get("schemeName", "")
        document += f"{'='*80}\n"
        document += f"{scheme_name.upper()}\n"
        document += f"{'='*80}\n\n"
        
        # Basic Details
        document += "SCHEME DETAILS\n"
        document += "-" * 80 + "\n\n"
        document += f"Scheme Name: {scheme_name}\n"
        document += f"Short Title: {basic_details.get('schemeShortTitle', '')}\n"
        
        state = basic_details.get("state", {}).get("label", "")
        if state:
            document += f"State: {state}\n"
        
        document += f"Level: {basic_details.get('level', {}).get('label', '')}\n"
        
        nodal_dept = basic_details.get("nodalDepartmentName", {}).get("label", "")
        if nodal_dept:
            document += f"Nodal Department: {nodal_dept}\n"
        
        # Categories
        categories = []
        for cat in basic_details.get("schemeCategory", []):
            cat_label = cat.get("label", "")
            if cat_label:
                categories.append(cat_label)
        
        if categories:
            document += f"Category: {', '.join(categories)}\n"
        
        sub_categories = []
        for sub_cat in basic_details.get("schemeSubCategory", []):
            sub_cat_label = sub_cat.get("label", "")
            if sub_cat_label:
                sub_categories.append(sub_cat_label)
        
        if sub_categories:
            document += f"Sub-Category: {', '.join(sub_categories)}\n"
        
        # Target beneficiaries
        beneficiaries = []
        for ben in basic_details.get("targetBeneficiaries", []):
            ben_label = ben.get("label", "")
            if ben_label:
                beneficiaries.append(ben_label)
        
        if beneficiaries:
            document += f"Target Beneficiaries: {', '.join(beneficiaries)}\n"
        
        # Tags
        tags = basic_details.get("tags", [])
        if tags:
            document += f"Tags: {', '.join(tags)}\n"
        
        # Implementing Agency
        impl_agency = basic_details.get("implementingAgency", "")
        if impl_agency:
            document += f"\nImplementing Agency:\n{impl_agency}\n"
        
        document += "\n"
        
        # Scheme Description
        document += "SCHEME DESCRIPTION\n"
        document += "-" * 80 + "\n\n"
        
        brief_desc = scheme_content.get("briefDescription", "")
        if brief_desc:
            document += f"{brief_desc}\n\n"
        
        detailed_desc = parse_content(scheme_content.get("detailedDescription", []))
        if detailed_desc:
            document += f"{detailed_desc}\n"
        
        # Benefits
        document += "BENEFITS\n"
        document += "-" * 80 + "\n\n"
        
        benefit_type = scheme_content.get("benefitTypes", {}).get("label", "")
        if benefit_type:
            document += f"Benefit Type: {benefit_type}\n\n"
        
        benefits = parse_content(scheme_content.get("benefits", []))
        if benefits:
            document += f"{benefits}\n"
        
        # Eligibility Criteria
        document += "ELIGIBILITY CRITERIA\n"
        document += "-" * 80 + "\n\n"
        
        eligibility_desc = parse_content(eligibility_criteria.get("eligibilityDescription", []))
        if eligibility_desc:
            document += f"{eligibility_desc}\n"
        
        # Exclusions
        exclusions = parse_content(scheme_content.get("exclusions", []))
        if exclusions:
            document += "EXCLUSIONS\n"
            document += "-" * 80 + "\n\n"
            document += f"{exclusions}\n"
        
        # Documents Required
        document += "DOCUMENTS REQUIRED\n"
        document += "-" * 80 + "\n\n"
        
        doc_required = parse_content(docs)
        if doc_required:
            document += f"{doc_required}\n"
        
        # Application Process
        document += "APPLICATION PROCESS\n"
        document += "-" * 80 + "\n\n"
        
        for app_process in application_process:
            mode = app_process.get("mode", "")
            url = app_process.get("url", "")
            
            if mode:
                document += f"Mode: {mode}\n"
            if url:
                document += f"URL: {url}\n\n"
            
            process_steps = parse_content(app_process.get("process", []))
            if process_steps:
                document += f"{process_steps}\n"
        
        # References
        references = scheme_content.get("references", [])
        if references:
            document += "REFERENCES\n"
            document += "-" * 80 + "\n\n"
            
            for ref in references:
                title = ref.get("title", "")
                url = ref.get("url", "")
                
                if title and url:
                    document += f"{title}: {url}\n"
            
            document += "\n"
        
        # FAQs
        if faqs:
            document += "FREQUENTLY ASKED QUESTIONS\n"
            document += "-" * 80 + "\n\n"
            
            for i, faq in enumerate(faqs, 1):
                question = faq.get("question", "")
                answer = parse_content(faq.get("answer", []))
                
                if question and answer:
                    document += f"Q{i}: {question}\n"
                    document += f"A{i}: {answer}\n"
        
        return document
    
    except Exception as e:
        return f"Error processing JSON: {str(e)}"

# Example usage
def test():
    # Read the JSON from a file
#     with open('jagananna_chedodu.json', 'r', encoding='utf-8') as file:
#         json_str = file.read()
    
    # Convert to human-readable document
    document = json_to_document(json_str)
    
    # Write the output to a file
#     with open('Jagananna_Chedodu_Scheme.txt', 'w', encoding='utf-8') as file:
#         file.write(document)
    
#     print("Successfully converted JSON to a human-readable document!")


In [12]:
def get_scheme_content(slug):
    url = "https://www.myscheme.gov.in/_next/data/Pgr1-v_XYCcKuy3LqoxeR/en/schemes/"+slug+".json?slug="+slug

    payload = {}
    headers = {
    'x-api-key': 'tYTy5eEhlu9rFjyxuCr7ra7ACp4dv1RH8gWuHTDc'
    }
#     print("calling scheme content with ",slug)
    response = requests.request("GET", url, headers=headers, data=payload)
#     print(response.headers)
    document = json_to_document(response.text)
    return document

In [14]:
print(get_scheme_content("faspalscf"))

FINANCIAL ASSISTANCE SCHEME FOR PURCHASE OF AGRICULTURAL LAND TO SCHEDULED CASTE FARMERS

SCHEME DETAILS
--------------------------------------------------------------------------------

Scheme Name: Financial Assistance Scheme for Purchase of Agricultural Land to Scheduled Caste Farmers
Short Title: FASPALSCF
State: Gujarat
Level: State/ UT
Nodal Department: Social Justice and Empowerment Department
Category: Banking,Financial Services and Insurance, Social welfare & Empowerment
Sub-Category: Financial assistance
Target Beneficiaries: Individual
Tags: Agriculture, Farmer, Financial Assistance, Purchase Of Land, Scheduled Caste

Implementing Agency:
District Deputy Director, Scheduled Caste Welfare Office, Gujarat

SCHEME DESCRIPTION
--------------------------------------------------------------------------------

The scheme was launched on in the year 1986. Under the scheme, financial assistance is provided to farmers belonging to the scheduled castes to purchase agricultural land, wi

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="keyhere")

In [ ]:
records=[
#     { "_id": "smcrcsc", "chunk_text": get_scheme_content("smcrcsc")}
]

for state in states:
    slugs=get_slugs(state['label'],str(state['count']))
    print("state ",state)
    print(len(slugs))
    for slug in slugs:
        records.append({"_id":slug,"chunk_text":get_scheme_content(slug)})

slugs=get_slugs("Andaman and Nicobar Islands","270")
for slug in slugs:
    records.append({"_id":slug,"chunk_text":get_scheme_content(slug)})

index_name="dense-index"
dense_index = pc.Index(name=index_name)

# Upsert records into a namespace
dense_index.upsert_records(
    namespace="govt_schemes", 
    records=records
)